In [1]:
import pandas as pd
from pyomo.environ import (ConcreteModel, Set, Param, Var, NonNegativeReals, Binary,
                           Objective, Constraint, minimize, value, SolverFactory)

In [2]:
import pandas as pd
from pyomo.environ import (ConcreteModel, Set, Param, Var, NonNegativeReals, Binary,
                           Objective, Constraint, minimize, value, SolverFactory)

def setup_model_LR(demand_df, supply_df, costs_df, lambda_values):

    # Define model
    model = ConcreteModel()

    # Sets
    model.S = Set(initialize=supply_df['Supply Site'].unique())
    model.D = Set(initialize=demand_df['Demand Site'].unique())
    model.T = Set(initialize=demand_df['Year'].unique())

    # Define parameters
    setup_costs = supply_df.set_index('Supply Site')['Setup Costs'].to_dict()
    model.SetupCost = Param(model.S, initialize=setup_costs)

    demand_data = demand_df.set_index(['Demand Site', 'Year'])['Demand'].to_dict()
    model.Demand = Param(model.D, model.T, initialize=demand_data)

    # Energy supply costs
    supply_costs = costs_df.set_index(['Supply Site', 'Demand Site', 'Year'])['Supply Cost'].to_dict()
    model.SupplyCost = Param(model.S, model.D, model.T, initialize=supply_costs)

    # Maximum Capacity parameter
    max_capacity = supply_df.set_index(['Supply Site'])['Capacity'].to_dict()
    model.MaxCapacity = Param(model.S, initialize=max_capacity)

    # Lagrange multipliers as parameters
    model.Lambda = Param(model.S, initialize=lambda_values)

    # Decision variables
    model.y = Var(model.S, domain=Binary)
    model.x = Var(model.S, model.D, model.T, domain=NonNegativeReals)

    def objective_rule(model):
        setup_cost = sum(model.SetupCost[s]*model.y[s] for s in model.S)
        supply_cost = sum(model.SupplyCost[s, d, t] * model.x[s, d, t]
                          for s in model.S for d in model.D for t in model.T)
        
        lagrangian_term = sum(model.Lambda[s] * 
                              (sum(model.x[s, d, t] for d in model.D for t in model.T) -
                              model.y[s] * model.MaxCapacity[s]) for s in model.S)
        return setup_cost + supply_cost + lagrangian_term

    model.Objective = Objective(rule=objective_rule, sense=minimize)

    # Constraints
    def demand_satisfaction_rule(model, d, t):
        return sum(model.x[s, d, t] for s in model.S) >= model.Demand[d, t]
    model.DemandSatisfaction = Constraint(model.D, model.T, rule=demand_satisfaction_rule)

    return model

def create_results_dfs(results, model, lambda_values, iterations):
    supply_results = []
    for s in model.S:
        for d in model.D:
            for t in model.T:
                energy_provided = value(model.x[s, d, t])
                if energy_provided > 0:
                    supply_results.append({
                        'Supply Site': s,
                        'Demand Site': d,
                        'Year': t,
                        'Energy Provided (MW)': energy_provided
                    })
    supply_results_df = pd.DataFrame(supply_results)

    constructed_sites = [s for s in model.S if model.y[s].value == 1]
    solve_time = results.solver.time if hasattr(results.solver, 'time') else None
    summary_df = pd.DataFrame({
        "Solve Time (seconds)": [solve_time],
        "Final Objective Value": [value(model.Objective)],
        "Number of Iterations": [iterations],
        "Lambda Values": [lambda_values],
        "Constructed Sites": [constructed_sites],
    })

    return supply_results_df, summary_df

def run_model_LR(demand_df, supply_df, costs_df, output_directory="", output_suffix="", model_solver='gurobi',
                 save_results=True, max_iterations=10, alpha0=100, beta=0.5):
    """
    Run Lagrangian relaxation using the r-algorithm to update Lagrange multipliers.
    
    Parameters
    ----------
    alpha0 : float
        Initial step size for subgradient updates.
    beta : float
        Weight parameter for combining lambda_values with u_values.
    """

    # Initialize lambda_values and u_values
    lambda_values = {s: 0.0 for s in supply_df['Supply Site'].unique()}
    u_values = lambda_values.copy()  # u_0 = lambda_0

    solver = SolverFactory(model_solver)

    for iteration in range(max_iterations):
        # Build and solve the LR model
        model = setup_model_LR(demand_df, supply_df, costs_df, lambda_values)
        results = solver.solve(model, tee=False)

        # Compute violations (subgradient)
        violations = {}
        lhs_dict = {}
        rhs_dict = {}
        for s in model.S:
            lhs = sum(value(model.x[s, d, t]) for d in model.D for t in model.T)
            rhs = value(model.y[s]) * value(model.MaxCapacity[s])
            violation = lhs - rhs
            violations[s] = violation
            lhs_dict[s] = lhs
            rhs_dict[s] = rhs

        # Compute step size alpha_k
        # For the r-algorithm, a diminishing step size is often used
        alpha_k = alpha0 / ((iteration + 1)**0.5)

        # Update u_values: u_{k+1} = u_k + alpha_k * g_k (where g_k = violations)
        for s in model.S:
            u_values[s] = u_values[s] + alpha_k * violations[s]

        # Update lambda_values using r-alg step:
        # lambda_{k+1} = lambda_k + beta * (u_{k+1} - lambda_k)
        for s in model.S:
            lambda_values[s] = lambda_values[s] + beta * (u_values[s] - lambda_values[s])

        # Check for convergence
        total_violation = sum(max(0, v) for v in violations.values())
        if total_violation < 1e-8:
            # Violations small enough, stop early
            break

    supply_results, summary_results = create_results_dfs(results, model, lambda_values, iteration)

    if save_results:
        supply_results.to_csv(f"{output_directory}supply_results_LR{output_suffix}.csv", index=False)
        summary_results.to_csv(f"{output_directory}summary_results_LR{output_suffix}.csv", index=False)

    return supply_results, summary_results, lambda_values, u_values


In [3]:
demand_df = pd.read_csv("Data/dummy_demand.csv")
supply_df = pd.read_csv("Data/dummy_supply.csv")
costs_df = pd.read_csv("Data/dummy_costs.csv")
run_model_LR(demand_df, supply_df, costs_df, "Outputs/", "_base_test_LR_r_algo_1")

(                                  Supply Site        Demand Site  Year  \
 0                        Glendoe Hydro Scheme     City of London  2005   
 1                        Glendoe Hydro Scheme     City of London  2006   
 2                        Glendoe Hydro Scheme     City of London  2007   
 3                        Glendoe Hydro Scheme     City of London  2008   
 4                        Glendoe Hydro Scheme     City of London  2009   
 5                        Glendoe Hydro Scheme     City of London  2010   
 6   Gate Burton - Solar & Energy Storage Park         Birmingham  2005   
 7   Gate Burton - Solar & Energy Storage Park         Birmingham  2006   
 8   Gate Burton - Solar & Energy Storage Park         Birmingham  2007   
 9   Gate Burton - Solar & Energy Storage Park         Birmingham  2008   
 10  Gate Burton - Solar & Energy Storage Park         Birmingham  2009   
 11  Gate Burton - Solar & Energy Storage Park         Birmingham  2010   
 12           Berwick Ban

In [6]:
costs_df_2 = pd.read_csv("Data/dummy_costs_2.csv")
run_model_LR(demand_df, supply_df, costs_df_2, "Outputs/", "_base_test_LR_r_algo_2")

model.name="unknown";
    - termination condition: unbounded
    - message from solver: Model was proven to be unbounded.
model.name="unknown";
    - termination condition: unbounded
    - message from solver: Model was proven to be unbounded.


(                         Supply Site        Demand Site  Year  \
 0   Berwick Bank Offshore Wind Farm      City of London  2005   
 1   Berwick Bank Offshore Wind Farm      City of London  2006   
 2   Berwick Bank Offshore Wind Farm      City of London  2007   
 3   Berwick Bank Offshore Wind Farm      City of London  2008   
 4   Berwick Bank Offshore Wind Farm      City of London  2009   
 5   Berwick Bank Offshore Wind Farm      City of London  2010   
 6   Berwick Bank Offshore Wind Farm   City of Edinburgh  2005   
 7   Berwick Bank Offshore Wind Farm   City of Edinburgh  2006   
 8   Berwick Bank Offshore Wind Farm   City of Edinburgh  2007   
 9   Berwick Bank Offshore Wind Farm   City of Edinburgh  2008   
 10  Berwick Bank Offshore Wind Farm   City of Edinburgh  2009   
 11  Berwick Bank Offshore Wind Farm   City of Edinburgh  2010   
 12  Berwick Bank Offshore Wind Farm          Birmingham  2005   
 13  Berwick Bank Offshore Wind Farm          Birmingham  2006   
 14  Berwi